<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/cda-2022/blob/main/notebooks/3_maxlik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment

### Python libraries

In [2]:
import scipy.stats as st
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## Setup R via Python

In [3]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("maxLik")

In [5]:
%%R
library(maxLik)

R[write to console]: Loading required package: miscTools

R[write to console]: 
Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/



## Setup Julia via Python

In [ ]:
%%bash
wget https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz
tar zxvf julia-1.7.2-linux-x86_64.tar.gz
## pythons module
pip install julia

In [7]:
import julia
julia.install(julia = "/content/julia-1.7.2/bin/julia")
from julia import Julia
jl = Julia(runtime="/content/julia-1.7.2/bin/julia",compiled_modules=False)
%load_ext julia.magic


Precompiling PyCall...
Precompiling PyCall... DONE
PyCall is installed and built successfully.

PyCall is setup for non-default Julia runtime (executable) `/content/julia-1.7.2/bin/julia`.
To use this Julia runtime, PyJulia has to be initialized first by
    from julia import Julia
    Julia(runtime='/content/julia-1.7.2/bin/julia')


Initializing Julia interpreter. This may take some time...


/usr/local/lib/python3.7/dist-packages/julia/magic.py:166: UserWarning: Incompatible upstream libraries. Got ImportError: No module named 'prompt_toolkit.lexers'
  warnings.warn(template.format(err))
/usr/local/lib/python3.7/dist-packages/julia/magic.py:174: UserWarning: Incompatible upstream libraries. Got ImportError: cannot import name 'Completion' from 'IPython.core.completer' (/usr/local/lib/python3.7/dist-packages/IPython/core/completer.py)
  warnings.warn(template.format(err))


In [ ]:
%%julia
using Pkg
Pkg.add("Distributions")
Pkg.add("DataFrames")
Pkg.add("Optim")
using Distributions
using DataFrames
using Random
using Optim
Pkg.status()

## Exercise -- zero-truncated Poisson distribution

We start with likelihood function

\begin{equation}
    L = \prod_i \frac{\lambda^x_i}{(e^\lambda-1)x_i!},
\end{equation}

then we compute log-likelihood

\begin{equation}
   \log L = \sum_i x_i \log \lambda - \sum_i \log(e^\lambda-1) - \sum_i \log(x_i!) 
\end{equation}

In order to get estimate of $\lambda$ we need to calculate derivatives with respect to this parameter. Thus, gradient is given by 

\begin{equation}
    \frac{\partial \log L}{\partial \lambda} = \frac{\sum_i x_i}{\lambda} - \frac{n e^\lambda}{e^\lambda - 1} = 
    \frac{\sum_i x_i}{\lambda} - n \frac{e^\lambda}{e^\lambda - 1}.    
\end{equation}

We can also calculate second derivative (hessian)

\begin{equation}
    \frac{\partial^2 \log L}{\partial \lambda^2} =  - \frac{\sum_i x_i}{\lambda^2} + n \frac{e^\lambda}{(e^\lambda-1)^2}.
\end{equation}


## Solution in R

In [9]:
%%R
## log-likelihood function
ll <- function(par, x) {
  m <- sum(x)*log(par)-length(x)*log(exp(par)-1)
  m
}

## gradient
grad <- function(par, x)  {
  g <- sum(x) / par - length(x)*exp(par)/(exp(par)-1)
  g
}


## hessian
hess <- function(par, x) {
  h <- -sum(x)/par^2 + length(x)*exp(par)/(exp(par)-1)^2 
  h
}

d <-  c(1645,183,37, 13,1,1)
x <- rep(1:6,d)

## with gradient and hessian
res2 <- maxLik(logLik = ll,  grad = grad, 
            hess = hess, start = 1, x = x, method = "NR")

summary(res2)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 6 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -656.1294 
1  free parameters
Estimates:
     Estimate Std. error t value Pr(> t)    
[1,]  0.30862    0.01726   17.88  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
--------------------------------------------


## Solution in Python

In [10]:
## python does minimization so we need to have -logL
def ll(par,x):
  m = np.sum(x)*np.log(par)-len(x)*np.log(np.exp(par)-1)
  return -m

## gradient
def grad(par,x):
  g = np.sum(x) / par - len(x)*np.exp(par)/(np.exp(par)-1)
  return -g

## hessian
def hess(par,x):
  h = -np.sum(x)/par**2 + len(x)*np.exp(par)/(np.exp(par)-1)**2 
  return h

d = np.array([1645,183,37, 13,1,1])
x = np.repeat(np.arange(1,7), d)
res = minimize(fun=ll, x0=[0.5], method = "Newton-CG", jac = grad, hess = hess, args = (x))
res

     fun: 656.1294299023266
     jac: array([-0.00187348])
 message: 'Optimization terminated successfully.'
    nfev: 7
    nhev: 5
     nit: 5
    njev: 11
  status: 0
 success: True
       x: array([0.30861895])

## Solution in Julia

In [15]:
%%julia
## logL - minimization
function ll(par, x)
  par = par[1]
  m = sum(x)*log(par)-length(x)*log(exp(par)-1)
  return -m
end


## gradient
function grad!(G,par,x) 
  par = par[1]
  G[1] = -(sum(x) / par - length(x)*exp(par)/(exp(par)-1))
  return G
end 

## hessian
function hess!(H,par, x)
  par = par[1]
  H[1] = -sum(x)/par^2 + length(x)*exp(par)/(exp(par)-1)^2 
  return H
end

<PyCall.jlwrap hess!>

In [16]:
%%julia
d = [1645,183,37, 13,1,1]
x = vcat(fill.(1:6, d)...)
fun_opt = TwiceDifferentiable(par -> ll(par, x), 
                              (G, par) -> grad!(G, par, x), 
                              (H, par) -> hess!(H, par, x), 
                              [0.5])

res = optimize(fun_opt, [0.5])

<PyCall.jlwrap  * Status: success

 * Candidate solution
    Final objective value:     6.561294e+02

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 1.06e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.44e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.96e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.98e-14 ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    9
    ∇f(x) calls:   9
    ∇²f(x) calls:  4
>

In [17]:
%%julia
Optim.minimizer(res)

array([0.30861895])